In [1]:
!rm -f .cache.db

In [3]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True,
)


In [4]:
import langchain
from langchain.cache import SQLiteCache

langchain.llm_cache = SQLiteCache(database_path=".cache.db")

In [6]:
%%time
# The first time, it is not yet in cache, so it should take longer
cf_llm_openai.invoke("Tell me a joke")

CPU times: user 31.7 ms, sys: 19.1 ms, total: 50.8 ms
Wall time: 54.4 ms


AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHow was that? Do you want to hear another one?", id='run-465fd061-b6a3-4fd5-a7d0-7c6613a59829-0')

In [7]:
%%time
# 在这个句子中添加一些空格然后再提问
cf_llm_openai.invoke("Tell me a   joke")
# 又会重新执行 
# Why does the extra space cause the cache miss??

CPU times: user 31.6 ms, sys: 6.7 ms, total: 38.3 ms
Wall time: 5.75 s


AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHow was that? Do you want to hear another one?", id='run-720a2ec6-b4ac-4e10-b01b-38a2b9bebb28-0')

In [8]:
import sqlalchemy
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///.cache.db")

In [9]:
with engine.connect() as connection:

    rs = connection.exec_driver_sql('select * from full_llm_cache')
    print(rs.keys())
    for row in rs:
        print(row)

RMKeyView(['prompt', 'llm', 'idx', 'response'])
('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "Tell me a joke", "type": "human"}}]', '{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "@cf/meta/llama-3-8b-instruct", ... (485 characters truncated) ... }, {"id": 2, "type": "schema", "data": "ChatOpenAIOutput"}], "edges": [{"source": 0, "target": 1}, {"source": 1, "target": 2}]}}---[(\'stop\', None)]', 0, '{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output", "ChatGeneration"], "kwargs": {"text": "Here\'s one:\\n\\nWhy couldn\'t the b ... (417 characters truncated) ... ? Do you want to hear another one?", "type": "ai", "id": "run-465fd061-b6a3-4fd5-a7d0-7c6613a59829-0", "tool_calls": [], "invalid_tool_calls": []}}}}')
('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": 